# Hacking `treelite` for Out-Of-Bag Random Forest Predictions

Random forests are great. So great that I implore you to read one of the thousand articles about random forests starting [here](https://en.wikipedia.org/wiki/Random_forest). Each tree in a random forest is constructed from independent samples of the source data (a la [bootstrap aggregating](https://en.wikipedia.org/wiki/Bootstrap_aggregating)). This leads to two interesting features:

* **Prediction performance**: Random forests generally do not overfit when one cranks up the number of trees while mostly enjoying variance reduction gains. Hence training a forest with a large number of trees is generally desirable albeit computationally expensive. Predicting through such a forest is also expensive. Fortunately, [treelite](https://treelite.readthedocs.io/en/latest/) takes care of prediction (and only prediction, not training) performance. Since any fancy (or plain), causal (or not), honest (or cheating) random forest are pretty much the same **at prediction time** -- bunch of split test nodes and leaf average values -- treelite can convert a bespoke representation of random forest into a fast C one. This works for xgboost, lightgbm, sklearn, or even your own hand rolled random forest model. The treelite representation of random forest can then be compiled into a quick C shared object or deployed to CUDA ([cuml](https://github.com/rapidsai/cuml)) / [Sagemaker Neo](https://github.com/neo-ai/neo-ai-dlr) for glory.
* **Out-of-bag errors**: Instead of having to do an explicit train test split, one can enjoy the convenience of having random forests calculate out-of-bag errors by making each individual tree predict on the subset of the data that was not used to train the tree. This is great and useful in a lot of practical applications. **However, this is not supported by `treelite`** as it requires each forest prediction call to only query from a subset of trees in the forest. Instead, the `treelite` interface only supports querying for an average value across all trees in the forest.

In this article, we'll see how we can (ab)use `treelite` to get out-of-bag predictions. Additionally, I hope that this will help make the `treelite` package more popular among causal circles.

## Getting OOB Predictions and Sampling Indices from `sklearn`

We first start with a simple `sklearn.ensemble.RandomForestRegressor` model trained on the `california_housing` dataset.

In [1]:
import sklearn.datasets
import sklearn.ensemble
import numpy
import treelite
import treelite_runtime

In [2]:
%cd treelite-oob

[Errno 2] No such file or directory: 'treelite-oob'
/home/jovyan/treelite-oob


In [3]:
d = sklearn.datasets.fetch_california_housing(data_home = './bin')

In [4]:
x = d['data']
y = d['target']
x.shape

(20640, 8)

In [5]:
forest = sklearn.ensemble.RandomForestRegressor(n_estimators = 100, max_depth = 3, oob_score = True, random_state = 0)
forest.fit(x, y)

RandomForestRegressor(max_depth=3, oob_score=True, random_state=0)

We can now query the model for its training score, oob score, and oob predictions.

In [6]:
print(forest.score(x, y), forest.oob_score_, len(forest.estimators_))
print(forest.oob_prediction_)

0.5613827712406585 0.5539804583170151 100
[4.26892639 4.48427396 3.62823672 ... 1.20096897 1.40188469 1.21079713]


Indeed, we can peel under the hood a little and see exactly how `oob_prediction_` is calculated. When one looks at the python source code for that, one can see that it's essentially calculating the following for a given tree `i`

* Get the observations that the tree `i` did not bootstrap sample (i.e. `unsampled_indices`)
* Have that tree `i` predict `x[unsampled_indices, :]` i.e. the part of the data that tree `i` did not see

Then, over all trees, average those predictions by the number of trees that contributed to any particular prediction.

I replicated the `oob_prediction_` code below for both clarity and since it'll be useful in the `treelite` reproduction of oob later.

In [7]:
# check that my reproduction of bootstrap sampling indices is accurate

oob_preds = numpy.zeros(y.shape[0])
n_oob_preds = numpy.zeros(y.shape[0])

for i in range(forest.n_estimators):
    tree = forest.estimators_[i]
    unsampled_indices = sklearn.ensemble._forest._generate_unsampled_indices(tree.random_state, y.shape[0], y.shape[0])
    oob_pred = tree.predict(x[unsampled_indices, :])
    oob_preds[unsampled_indices] += oob_pred
    n_oob_preds[unsampled_indices] += 1

n_oob_preds[n_oob_preds == 0] = 1
oob_preds /= n_oob_preds

In [8]:
numpy.all(oob_preds == forest.oob_prediction_)

True

## Converting `sklearn` Model to `treelite` (Vanilla)

Now we can convert this `sklearn.ensemble.RandomForestRegressor` to a `treelite` model. `treelite` comes with a converter for `sklearn` models so we use that directly.

In [9]:
model_sklearn = treelite.sklearn.import_model(forest)
model_sklearn.export_lib(toolchain = 'gcc', libpath = './bin/model_sklearn.so', params = {'parallel_comp': 32})
predictor_sklearn = treelite_runtime.Predictor('./bin/model_sklearn.so')

/opt/conda/lib/python3.10/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


[03:05:29] ../src/compiler/ast/split.cc:29: Parallel compilation enabled; member trees will be divided into 32 translation units.


In [10]:
pred_truth = forest.predict(x)
pred_sklearn = predictor_sklearn.predict(treelite_runtime.DMatrix(x, dtype = 'float32'), verbose = True)

numpy.allclose(pred_truth, pred_sklearn)

[03:05:31] ../src/predictor/predictor.cc:464: Treelite: Finished prediction in 0.00171971 sec


True

We find that all the predictions closely match up. Excellent.

**However, we have lost the ability to make OOB predictions**. Oh no woe be us. Fortunately, `treelite` provides us the knobs to restore this ability.

First, let's leave the precanned `treelite.sklearn.import_model` adapter behind and instead construct the `treelite` model from the `sklearn` model by hand. We do so by literally following the `treelite` tutorial in the docs.

## Converting `sklearn` Model to `treelite` (By Hand)

In order to do this, we basically loop through each tree in the forest and traverse the `sklearn` tree structure iteratively. Most of the hard part is figuring out the `sklearn` API for each individual tree. The guys over at `treelite` did an excellent job with the tutorial and I just copied their code over.

In [11]:
def process_node(treelite_tree, sklearn_tree, node_id, sklearn_model):
    if sklearn_tree.children_left[node_id] == -1:  # leaf node
        process_leaf_node(treelite_tree, sklearn_tree, node_id, sklearn_model)
    else:  # test node
        process_test_node(treelite_tree, sklearn_tree, node_id, sklearn_model)

def process_test_node(treelite_tree, sklearn_tree, node_id, sklearn_model):
    # Process a test node with a given node ID. We shall assume that all tree ensembles in
    # scikit-learn use only numerical splits.
    treelite_tree[node_id].set_numerical_test_node(
        feature_id = sklearn_tree.feature[node_id],
        opname = '<=',
        threshold = sklearn_tree.threshold[node_id],
        threshold_type = 'float64',
        default_left = True,
        left_child_key = sklearn_tree.children_left[node_id],
        right_child_key = sklearn_tree.children_right[node_id]
    )

def process_leaf_node(treelite_tree, sklearn_tree, node_id, sklearn_model):
    # Process a test node with a given node ID
    # The `value` attribute stores the output for every leaf node.
    leaf_value = sklearn_tree.value[node_id].squeeze()
    # Initialize the leaf node with given node ID
    treelite_tree[node_id].set_leaf_node(leaf_value, leaf_value_type = 'float64')


builder_hand = treelite.ModelBuilder(num_feature = forest.n_features_in_, average_tree_output = True, threshold_type = 'float64', leaf_output_type = 'float64')
for i in range(forest.n_estimators):
    sklearn_tree = forest.estimators_[i].tree_
    treelite_tree = treelite.ModelBuilder.Tree(threshold_type = 'float64', leaf_output_type = 'float64')
    
    for node_id in range(sklearn_tree.node_count):
        process_node(treelite_tree, sklearn_tree, node_id, sklearn_model = forest)
    treelite_tree[0].set_root()
    builder_hand.append(treelite_tree)

model_hand = builder_hand.commit()

In [12]:
model_hand.export_lib(toolchain = 'gcc', libpath = './bin/model_hand.so', params = {'parallel_comp': 32})
predictor_hand = treelite_runtime.Predictor('./bin/model_hand.so')

[03:05:31] ../src/compiler/ast/split.cc:29: Parallel compilation enabled; member trees will be divided into 32 translation units.


In [13]:
print(predictor_hand.leaf_output_type)

pred_truth = forest.predict(x)
pred_hand = predictor_hand.predict(treelite_runtime.DMatrix(x, dtype = 'float32'), verbose = True)

numpy.allclose(pred_truth, pred_hand)

float64
[03:05:34] ../src/predictor/predictor.cc:464: Treelite: Finished prediction in 0.00194001 sec


True

The hand constructed `treelite` model matches the original `sklearn` random forest predictions as well. That's great.

Now let's add in the fun out-of-bag predictions.

## Adding Out-Of-Bag Predictions to `treelite`

Let's look at our random forest again. We have `ntree = 100` trees in the random forest. When we predict through `m` rows, we get a `m` length output: each element is a prediction for the `i`-th row of data. That's simple enough.

Suppose we were instead able to get back a `m * ntree` matrix where

* Each row `i` in `m` corresponds to the predictions for a single row in the prediction data (same as before)
* Each column `j` in `ntree` corresponds to tree `j`'s prediction for that row.

In other words, `prediction[i, j]` would mean that for the `i`-th observation, tree `j` predicted `prediction[i, j]`. In order to get the average prediction (which is what we would do when predicting), we can take `avg(prediction[i, :])`. 

If we wanted to predict out-of-bag, then we would simply subset `prediction[i, :]` to the specific trees that did not see observation `i` in the original training data.

* For example, our current data has `x.shape = (20640, 8)`.
* Suppose tree `5` was trained on all even observations (i.e. observations `[0, 2, 4, 6, 8, ...]`
* Then, the tree `5`'s predictions can only be used to predict observations `[1, 3, 5, 7, 9, ...]`
* So the out-of-bag prediction for row `1` should contain tree `5`'s prediction for row `1`.

Then, **we just have to get `treelite` to return an array instead of a single average value at its leaf nodes!** Turns out that's possible because `treelite` supports [multi-class classification](https://treelite.readthedocs.io/en/latest/tutorials/builder.html#multi-class-classification-with-randomforestclassifier) and we can (ab)use that interface to return an array at leaf nodes.

We do this below. Key tips:

* Make sure that the `treelite` tree is constructed using `pred_transform = 'identity_multiclass'` so that the prediction values are preserved (vs. be converted into a classification prediction
* The leaf vector is `zeros` except for the element at the `tree_index`. i.e. tree `1`'s vector is `[pred_1, 0, 0, 0, 0, ...]`, tree `2`'s vector is `[pred_2, 0, 0, 0, 0, ...]`. Then, when `treelite` finally averages the vectors column wise for a prediction, the resulting vector will be `[pred_1 / ntree, pred_2 / ntree, pred_3 / ntree, ...]`. Do remember to multiply the values back by `ntree`

In [14]:
def process_node_oob(treelite_tree, sklearn_tree, node_id, sklearn_model, tree_index):
    if sklearn_tree.children_left[node_id] == -1:  # leaf node
        process_leaf_node_oob(treelite_tree, sklearn_tree, node_id, sklearn_model, tree_index)
    else:  # test node
        process_test_node_oob(treelite_tree, sklearn_tree, node_id, sklearn_model)

def process_test_node_oob(treelite_tree, sklearn_tree, node_id, sklearn_model):
    # Process a test node with a given node ID. We shall assume that all tree ensembles in
    # scikit-learn use only numerical splits.
    treelite_tree[node_id].set_numerical_test_node(
        feature_id = sklearn_tree.feature[node_id],
        opname = '<=',
        threshold = sklearn_tree.threshold[node_id],
        threshold_type = 'float64',
        default_left = True,
        left_child_key = sklearn_tree.children_left[node_id],
        right_child_key = sklearn_tree.children_right[node_id]
    )

def process_leaf_node_oob(treelite_tree, sklearn_tree, node_id, sklearn_model, tree_index):
    # Process a test node with a given node ID
    # The `value` attribute stores the output for every leaf node.
    leaf_value = sklearn_tree.value[node_id].squeeze()
    # Initialize the leaf node with given node ID
    # use a vector by tree index
    leaf_vector = numpy.zeros(sklearn_model.n_estimators)
    leaf_vector[tree_index] = leaf_value
    treelite_tree[node_id].set_leaf_node(leaf_vector, leaf_value_type = 'float64')


builder_oob = treelite.ModelBuilder(num_feature = forest.n_features_in_, average_tree_output = True, 
                                    pred_transform = 'identity_multiclass', num_class = forest.n_estimators,
                                    threshold_type = 'float64', leaf_output_type = 'float64')
for i in range(forest.n_estimators):
    sklearn_tree = forest.estimators_[i].tree_
    treelite_tree = treelite.ModelBuilder.Tree(threshold_type = 'float64', leaf_output_type = 'float64')
    
    for node_id in range(sklearn_tree.node_count):
        process_node_oob(treelite_tree, sklearn_tree, node_id, sklearn_model = forest, tree_index = i)
    treelite_tree[0].set_root()
    builder_oob.append(treelite_tree)

model_oob = builder_oob.commit()

In [15]:
model_oob.export_lib(toolchain = 'gcc', libpath = './bin/model_oob.so', params = {'parallel_comp': 32})
predictor_oob = treelite_runtime.Predictor('./bin/model_oob.so')

[03:05:35] ../src/compiler/ast/split.cc:29: Parallel compilation enabled; member trees will be divided into 32 translation units.


In [16]:
print(predictor_oob.leaf_output_type)

pred_truth = forest.predict(x)
pred_oob = predictor_oob.predict(treelite_runtime.DMatrix(x, dtype = 'float32'), verbose = True)

float64
[03:05:45] ../src/predictor/predictor.cc:464: Treelite: Finished prediction in 0.0262787 sec


In [17]:
pred_oob

array([[0.04560552, 0.04570032, 0.04586979, ..., 0.0412651 , 0.04568925,
        0.04634979],
       [0.04560552, 0.04570032, 0.04586979, ..., 0.0412651 , 0.04568925,
        0.04634979],
       [0.0370852 , 0.03759835, 0.03906212, ..., 0.02751148, 0.03786934,
        0.03836738],
       ...,
       [0.01124395, 0.01140075, 0.01148075, ..., 0.01163021, 0.01270512,
        0.0117942 ],
       [0.01124395, 0.01140075, 0.01148075, ..., 0.01163021, 0.01895014,
        0.0117942 ],
       [0.01124395, 0.01140075, 0.01148075, ..., 0.01163021, 0.01270512,
        0.0117942 ]])

Let's take a look at the `pred_oob` output to see what we're dealing with.

* The first row (`pred_oob[0, :]`) corresponds to the first row of the training data (`x[0, :]`)
* The first element of the first row (`pred_oob[0, 0]`) is the prediction made by tree `0` for `x[0, :]`. That means the first tree predicted `0.04560552` for `x[0, :]`, the second tree predicted `0.04570032` for `x[0, :]` etc.
* The out-of-bag prediction for `x[0, :]` is then the subset of `pred_oob[0, :]` produced by trees that have not seen `x[0, :]`.

Hmm, where have we seen that `unsampled_indices` by tree before? Oh right! In the `sklearn` OOB reproduction at the top! We reproduce this below with a slightly different objective: to produce a boolean matrix `ntree * m` where `m = x.shape[0]` (rows of observations) where an element is true of a tree has seen an observation.

In [18]:
sampled_by_tree = numpy.ones((forest.n_estimators, y.shape[0]), dtype = 'bool')
for i in range(forest.n_estimators):
    tree = forest.estimators_[i]
    unsampled_indices = sklearn.ensemble._forest._generate_unsampled_indices(tree.random_state, y.shape[0], y.shape[0])
    sampled_by_tree[i, unsampled_indices] = 0

print(sampled_by_tree.shape)
sampled_by_tree

(100, 20640)


array([[False, False,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True, False,  True],
       [ True, False,  True, ..., False,  True, False],
       ...,
       [False, False, False, ...,  True,  True,  True],
       [ True, False, False, ..., False,  True, False],
       [False, False, False, ...,  True,  True,  True]])

Let's take a look at the first column of `sampled_by_tree`

In [19]:
# first observation were sampled by these trees
sampled_by_tree[:, 0]

array([False,  True,  True,  True, False, False,  True,  True, False,
       False,  True,  True,  True,  True, False,  True,  True, False,
        True, False, False,  True,  True,  True, False,  True,  True,
        True,  True,  True, False, False,  True,  True,  True,  True,
       False, False,  True, False,  True,  True,  True,  True,  True,
       False,  True, False, False,  True,  True, False,  True,  True,
       False,  True,  True, False,  True, False,  True, False, False,
       False,  True,  True,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True,  True, False, False, False,
        True,  True, False,  True,  True,  True,  True, False,  True,
       False])

This means that the first observation was

* NOT sampled by tree `0, 4, 5, ...` during training
* Sampled by tree `1, 2, 3, ...` during training

We can use this information to subset `pred_oob` to the predictions made by trees that did not sample `x[0, :]`.

First, we get the prediction made by all trees for `x[0, :]`

In [20]:
# prediction for first observation by these trees
pred_oob.T[:, 0]

array([0.04560552, 0.04570032, 0.04586979, 0.04569512, 0.03821047,
       0.04371979, 0.04589362, 0.0402859 , 0.04581989, 0.04577618,
       0.0400397 , 0.04544045, 0.03926476, 0.04001247, 0.04592836,
       0.04122305, 0.0412437 , 0.03999852, 0.04589456, 0.0392282 ,
       0.04590753, 0.04582622, 0.03946538, 0.0397702 , 0.04303944,
       0.04600728, 0.04095347, 0.04560237, 0.04599909, 0.04586869,
       0.03972882, 0.03990556, 0.04738166, 0.04646981, 0.03913978,
       0.04602887, 0.04108574, 0.04012991, 0.04545299, 0.03930916,
       0.04622483, 0.04627873, 0.04550603, 0.04582898, 0.0462196 ,
       0.04567977, 0.04682332, 0.04071481, 0.04121475, 0.04610376,
       0.04565651, 0.04593056, 0.0458457 , 0.04548754, 0.04324523,
       0.04683947, 0.03986283, 0.04567417, 0.04694492, 0.04606315,
       0.04620698, 0.03963573, 0.0394904 , 0.0390622 , 0.03992085,
       0.04229458, 0.04585559, 0.04605572, 0.04589857, 0.03961649,
       0.04561612, 0.04726357, 0.04569323, 0.04021279, 0.04561

Subsetting them to `sampled_by_tree[:, 0]` gives us the following smaller set of predictions. These are the out-of-bag predictions for `x[0, :]`, albeit scaled down because we divided through `ntree.`

In [23]:
pred_oob.T[:, 0][numpy.invert(sampled_by_tree[:, 0])]

array([0.04560552, 0.03821047, 0.04371979, 0.04581989, 0.04577618,
       0.04592836, 0.03999852, 0.0392282 , 0.04590753, 0.04303944,
       0.03972882, 0.03990556, 0.04108574, 0.04012991, 0.03930916,
       0.04567977, 0.04071481, 0.04121475, 0.04593056, 0.04324523,
       0.04567417, 0.04606315, 0.03963573, 0.0394904 , 0.0390622 ,
       0.04605572, 0.04589857, 0.04586562, 0.0397793 , 0.04042515,
       0.03957632, 0.04683772, 0.0419671 , 0.0412651 , 0.04634979])

Taking the average of these and scaling them back up gives us the out-of-bag prediction for `x[0, :]`

In [21]:
numpy.mean(pred_oob.T[:, 0][numpy.invert(sampled_by_tree[:, 0])]) * forest.n_estimators

4.268926388720941

We can do this for all of `x` and we check that it's indeed equal to `sklearn`'s `oob_prediction_`.

In [22]:
pred_oob_mean = numpy.array([numpy.mean(pred_oob.T[:, i][numpy.invert(sampled_by_tree[:, i])]) * forest.n_estimators for i in range(y.shape[0])])
numpy.allclose(forest.oob_prediction_, pred_oob_mean)

True

We now have blazingly fast out of bag predictions for random forests.